<a href="https://colab.research.google.com/github/ss1111119/youtube-audio-to-text-srt-converter/blob/main/YouTube_Audio_to_Text_%26_SRT_Converter_(YouTube_%E9%9F%B3%E8%A8%8A%E8%BD%89%E6%96%87%E5%AD%97%E8%88%87%E5%AD%97%E5%B9%95%E8%BD%89%E6%8F%9B%E5%99%A8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  步驟 1：設定 Colab 執行環境

1. 在 Colab 中，選擇 **"Runtime"** > **"Change runtime type"**，然後在 **"Hardware accelerator"** 選擇 **"GPU"**，並確保選擇的是 **T4 GPU**。這樣可以加速後續的語音辨識處理。

2. 接下來，您需要安裝以下兩個必要的 Python 庫：
   - **Whisper**：由 OpenAI 提供的語音辨識模型。
   - **yt-dlp**：用來下載 YouTube 影片的音訊。


In [ ]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install yt-dlp

步驟 2：使用教學

### 1.	下載 YouTube 影片音訊

您可以在程式碼中輸入您想要處理的 YouTube 影片 URL。程式會使用 yt-dlp 下載影片音訊並提取音訊檔案。

### 2.	選擇 Whisper 模型

您將會被要求選擇想要使用的 Whisper 模型，模型的選擇會影響語音辨識的精度和速度。可選擇的模型包括：

	•	tiny
	•	base
	•	small
	•	medium
	•	large
	•	large-v2
	•	large-v3

### 3.	選擇保存檔案格式
您可以選擇是否將辨識結果保存為文字檔（.txt）或字幕檔（.srt）。程式會根據您的選擇生成並保存相應的檔案。

### 4.	執行語音辨識
程式會自動使用 Whisper 模型對下載的音訊進行語音辨識，並將結果保存在指定格式的檔案中。您將能夠下載辨識結果檔案。

### 5.	清理與釋放記憶體
語音辨識完成後，程式會刪除音訊檔案並清理 GPU 快取，釋放資源。

範例流程

	•	輸入 YouTube 影片 URL。
	•	選擇 Whisper 模型（例如 large）。
	•	選擇是否保存為文字檔和/或字幕檔。
	•	執行程式後，等待語音辨識完成並下載結果檔案。

這樣，您就完成了 YouTube 影片的語音辨識，並將結果保存為您選擇的格式。

In [ ]:
import os
import torch
import whisper
from pathlib import Path
from google.colab import files
import yt_dlp

# 設定裝置
device = "cuda" if torch.cuda.is_available() else "cpu"

# 載入Whisper模型
def load_model(model_size):
    return whisper.load_model(model_size).to(device)

# 下載YouTube影片的音訊
def download_audio_from_youtube(url, out_dir="."):
    ydl_opts = {
        'format': 'bestaudio/best',
        'extractaudio': True,
        'audioquality': 1,
        'outtmpl': f'{out_dir}/%(id)s.%(ext)s',
        'quiet': False,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=True)
        file_name = ydl.prepare_filename(info_dict)
        video_title = info_dict.get('title', 'Untitled')  # 獲取影片標題
    return file_name, video_title

# 進行語音辨識
def transcribe_audio(model, file, text=True, srt=True, title=None):
    file_path = Path(file)
    output_directory = file_path.parent

    # 使用Whisper進行語音辨識
    result = model.transcribe(file, verbose=False)

    # 生成並保存文字檔 (.txt)
    if text:
        txt_path = file_path.with_suffix(".txt")
        with open(txt_path, "w", encoding="utf-8") as txt:
            txt.write(result["text"])

    # 生成並保存字幕檔 (.srt)
    if srt:
        srt_writer = whisper.utils.get_writer("srt", output_directory)
        srt_content = result["segments"]

        # 在srt檔案最前面加入影片標題
        srt_content.insert(0, {'start': 0, 'end': 0, 'text': f"Title: {title}\n\n"})

        # 使用影片標題來命名字幕檔
        srt_filename = f"{title}.srt"
        srt_writer({"segments": srt_content}, srt_filename)

    # 下載完成後刪除音訊檔
    if file_path.exists():
        os.remove(file_path)
        print(f"音訊檔已被刪除：{file_path}")

    # 顯示已保存的檔案
    print(f"檔案已保存：{title}.txt 和 {title}.srt")
    return result

# 使用者輸入
YouTube_URL = input("請輸入YouTube影片的URL：")
whisper_model = input("請選擇Whisper模型（tiny, base, small, medium, large, large-v2, large-v3）：")
text = input("是否保存為文字檔（.txt）？(yes/no)：").strip().lower() == 'yes'
srt = input("是否保存為字幕檔（.srt）？(yes/no)：").strip().lower() == 'yes'

# 下載音訊並進行語音辨識
audio, video_title = download_audio_from_youtube(YouTube_URL)
model = load_model(whisper_model)
result = transcribe_audio(model, audio, text, srt, video_title)

del model  # 刪除模型
torch.cuda.empty_cache()  # 清空CUDA缓存